In [1]:
## Tracing the code execution with lang smith
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_API_KEY']="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"
os.environ['LANGCHAIN_PROJECT']="lang graph"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"

In [3]:
## Ne04j credentials
NEO4J_URI = "neo4j+s://c4ce356c.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "HwCyeI-JYu1simYHLbqUXL_emE_O42dMV6PRE2StBSk"

In [4]:
## Groq API key for accessing bigger models
groq_api='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'

In [5]:
## Initialize graph database
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [7]:
## Initialize LLM  
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api,model_name="llama-3.1-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000203A21FA240>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000203A21FADE0>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
## Import necessary library to load data and preprocess the data
from llama_index.core import SimpleDirectoryReader
from langchain_core.documents import Document
from langchain.docstore.document import Document
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
## Initialize file path from where the data is to be loaded
directory_path = "../dataset/extracted_best/"
## load all the markdownfiles
markdown_files = SimpleDirectoryReader(input_files=["c:/Users/s.vijaykumar/Desktop/skanda/bot/dataset/extracted_best/AMM1.md"]).load_data()
## Restructure the data format
# empty variable to store the processed data
new_documents = []
for document in markdown_files:
    page_content = document.text
    new_document = Document(page_content=page_content)## I removed the metadata to reduce for this instance
    new_documents.append(new_document)
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "","[","]"
]
## tokenizer model
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-en-icl")
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=200,
    chunk_overlap=30,
    strip_whitespace=True,
    separators=MARKDOWN_SEPARATORS
)
## chunkning the documents into smaller size
from tqdm import tqdm
documents3 = text_splitter.split_documents(new_documents)

In [13]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
## Initilize class to transform the data into nodes and relationships 
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents3)

In [14]:
len(graph_documents)

22

In [15]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Amm Connector', type='Connector', properties={}), Node(id='C R E A T I V E', type='Concept', properties={}), Node(id='I N T E R C O N N E C T', type='Concept', properties={}), Node(id='S O L U T I O N S', type='Concept', properties={}), Node(id='1 Mm Pitch', type='Attribute', properties={}), Node(id='Irresistibly Small', type='Attribute', properties={}), Node(id='Keep Your Connection Simple', type='Attribute', properties={}), Node(id='Available Off The Shelf', type='Attribute', properties={})], relationships=[Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=Node(id='C R E A T I V E', type='Concept', properties={}), type='RELATED_TO', properties={}), Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=Node(id='I N T E R C O N N E C T', type='Concept', properties={}), type='RELATED_TO', properties={}), Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=N

In [18]:
graph_documents[0].nodes

[Node(id='Amm Connector', type='Connector', properties={}),
 Node(id='C R E A T I V E', type='Concept', properties={}),
 Node(id='I N T E R C O N N E C T', type='Concept', properties={}),
 Node(id='S O L U T I O N S', type='Concept', properties={}),
 Node(id='1 Mm Pitch', type='Attribute', properties={}),
 Node(id='Irresistibly Small', type='Attribute', properties={}),
 Node(id='Keep Your Connection Simple', type='Attribute', properties={}),
 Node(id='Available Off The Shelf', type='Attribute', properties={})]

In [19]:
graph_documents[0].relationships

[Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=Node(id='C R E A T I V E', type='Concept', properties={}), type='RELATED_TO', properties={}),
 Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=Node(id='I N T E R C O N N E C T', type='Concept', properties={}), type='RELATED_TO', properties={}),
 Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=Node(id='S O L U T I O N S', type='Concept', properties={}), type='RELATED_TO', properties={}),
 Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=Node(id='1 Mm Pitch', type='Attribute', properties={}), type='HAS_ATTRIBUTE', properties={}),
 Relationship(source=Node(id='Amm Connector', type='Connector', properties={}), target=Node(id='Irresistibly Small', type='Attribute', properties={}), type='HAS_ATTRIBUTE', properties={}),
 Relationship(source=Node(id='Amm Connector', type='Connector', properties={}

In [20]:
## Loading the extracted transformed graph into the neo4j graph database
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [22]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
## Visualize the graph database using 3rd party widges
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [23]:
# ## Experimentation with open available data
# movies_query = """
# LOAD CSV WITH HEADERS FROM 
# 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
# AS row
# MERGE (c:Movie {id:row.movieId})
# SET m.released = date(row.released),
#     m.title = row.title,
#     m.imdbRating = toFloat(row.imdbRating)
# FOREACH (director in split(row.director, '|') | 
#     MERGE (p:Person {name:trim(director)})
#     MERGE (p)-[:DIRECTED]->(m))
# FOREACH (actor in split(row.actors, '|') | 
#     MERGE (p:Person {name:trim(actor)})
#     MERGE (p)-[:ACTED_IN]->(m))
# FOREACH (genre in split(row.genres, '|') | 
#     MERGE (g:Genre {name:trim(genre)})
#     MERGE (m)-[:IN_GENRE]->(g))
# """
# graph.query(movies_query)
# graph.refresh_schema()
# print(graph.schema)

In [24]:
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
## Define a cypher chain which helps in retrieval in graph database 
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x00000203F12EA690>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000203A21FA240>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000203A21FADE0>, model_name=

In [30]:
## Query inference
response = chain.invoke({"query": "What are the target market?"})
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Generated Cypher:
MATCH (n:Concept)-[:INCLUDES]->(m:Industry) RETURN m
Full Context:
[{'m': {'id': 'Civil Harsh Defence'}}, {'m': {'id': 'Harsh Industrial'}}, {'m': {'id': 'Avionics Industrial & Security'}}]

> Finished chain.


{'query': 'What are the target market?',
 'result': 'The target markets are Civil Harsh Defence, Harsh Industrial, and Avionics Industrial & Security.'}